In [1]:
# Colab settings
!wget --header="Host: archive.ics.uci.edu" --header="User-Agent: Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.88 Safari/537.36" --header="Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9" --header="Accept-Language: ru-RU,ru;q=0.9,en-US;q=0.8,en;q=0.7" --header="Referer: https://archive.ics.uci.edu/ml/machine-learning-databases/00363/" "https://archive.ics.uci.edu/ml/machine-learning-databases/00363/Dataset.zip" -O "Dataset.zip" -c
!unzip Dataset.zip
!git clone https://github.com/janchk/ML_HOMEWORK1
from ML_HOMEWORK1.src.models import LinearRegressionWithGd
from ML_HOMEWORK1.src.common import *

--2020-02-13 17:30:43--  https://archive.ics.uci.edu/ml/machine-learning-databases/00363/Dataset.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 19055526 (18M) [application/x-httpd-php]
Saving to: ‘Dataset.zip’

Dataset.zip         100%[===================>]  18.17M  19.8MB/s    in 0.9s    

2020-02-13 17:30:44 (19.8 MB/s) - ‘Dataset.zip’ saved [19055526/19055526]

Archive:  Dataset.zip
   creating: Dataset/
  inflating: Dataset/.DS_Store       
   creating: __MACOSX/
   creating: __MACOSX/Dataset/
  inflating: __MACOSX/Dataset/._.DS_Store  
  inflating: Dataset/Catagory_File - Feature 4.pdf  
  inflating: __MACOSX/Dataset/._Catagory_File - Feature 4.pdf  
   creating: Dataset/Testing/
  inflating: Dataset/Testing/.DS_Store  
   creating: __MACOSX/Dataset/Testing/
  inflating: __MACOSX/Dataset/Testing/._.DS_Store  
  

In [0]:
#import part 
import matplotlib.pyplot as plt
from IPython import display
# from src.models import LinearRegressionWithGd
# from src.common import *
import pandas as pd
path = "Dataset/Training/"

In [0]:
# configuration
num_folds = 5
epochs = 1000
learning_rate = 1e-3

lrg = LinearRegressionWithGd()
lrg.learning_rate = learning_rate

df_1, df_2 = folding(path)
np.random.shuffle(df_1)

### Training

In [7]:
features = []
bias = []
stats = []
losses = [] #loss per fold
for i in range(num_folds):
    train, test = cv_loo(df_1, num_folds, i)

    X_train = normalized(train.T[0:53].T)
    X_test = normalized(test.T[0:53].T)

    Y_train = train.T[53].T
    Y_train = np.expand_dims(Y_train, axis=-1)

    Y_test = test.T[53].T
    Y_test = np.expand_dims(Y_test, axis=-1)

    loss = lrg.train(epochs, X_train, Y_train)
    stat = lrg.validate(X_test, Y_test)
    losses.append(loss)
    stats.append(stat)
    features.append(lrg.betas)
    bias.append(*lrg.bias)
    

mean_stats = np.sum(stats, axis=0) / num_folds
print("mse_val is {}, r2_val is {}, rmse is {}, r2/rmse {}".format(
    mean_stats[0], mean_stats[1], mean_stats[2], mean_stats[3]))


Loss: 824.5289:   0%|          | 2/1000 [00:00<00:50, 19.59it/s]

val loss is 563.167709900831, R2 is 0.3927794934057012, RMSE is 23.731154837066633, R2/RMSE is 0.016551217001551197


Loss: 900.8882:   0%|          | 2/1000 [00:00<00:50, 19.94it/s]

val loss is 1118.6737348756358, R2 is 0.29374966848665085, RMSE is 33.446580316612874, R2/RMSE is 0.008782651789987205


Loss: 835.8405:   0%|          | 3/1000 [00:00<00:48, 20.53it/s]

val loss is 815.4430268191686, R2 is 0.3091650683731676, RMSE is 28.555963069369042, R2/RMSE is 0.010826637771667308


Loss: 884.0796:   0%|          | 0/1000 [00:00<?, ?it/s]

val loss is 1083.3847656253267, R2 is 0.295700999591313, RMSE is 32.914810733548606, R2/RMSE is 0.008983828039755918


Loss: 878.4064: 100%|██████████| 1000/1000 [00:49<00:00, 20.22it/s]


val loss is 881.4466831550576, R2 is 0.3045791984233345, RMSE is 29.689167774713013, R2/RMSE is 0.010258933518599736
mse_val is 892.423184075204, r2_val is 0.3191948856560335, rmse is 29.667535346262035, r2/rmse 0.011080653624312273


### Features

In [8]:
columns = ["bias"] + [f"f{f+1}" for f in range(len(features[0]))]
index = [f"Fold {i+1}" for i, _ in enumerate(features)] + ["mean"] + ["std"]

bias_np = np.expand_dims(np.squeeze(bias), 1)
features_np = np.squeeze(features, axis=2)
features_np = np.hstack((bias, features_np))
features_mean = np.mean(features_np, axis=0)
features_std = np.std(features_np, axis=0)
df = pd.DataFrame(np.vstack((features_np, features_mean, features_std )),
                  columns=columns,
                  index=index)

display.display(df)    

,bias,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12,f13,f14,f15,f16,f17,f18,f19,f20,f21,f22,f23,f24,f25,f26,f27,f28,f29,f30,f31,f32,f33,f34,f35,f36,f37,f38,f39,f40,f41,f42,f43,f44,f45,f46,f47,f48,f49,f50,f51,f52,f53
Fold 1,7.294475,0.395839,-0.096299,-1.460429,0.055669,-0.971484,0.834369,2.077066,1.556115,3.397266,1.983012,-1.119421,-0.411491,6.288889,-0.890636,-0.447877,0.792554,-0.481599,-0.412024,0.133312,-1.315610,0.021517,-0.426100,1.134811,0.288421,1.607342,-0.600967,0.907121,-1.722465,-0.859328,-0.862891,10.025607,3.301377,-1.735529,6.817985,-4.077646,0.041734,2.374424,0.909436,0.923781,0.562549,0.647912,0.937701,0.988534,0.625553,0.696607,0.536298,0.924596,1.183344,1.037183,1.093827,0.602390,0.898531,0.950712
Fold 2,7.162263,0.237424,-0.082128,-1.353467,-0.004047,-0.300492,0.910094,3.079327,1.089313,4.571588,1.428987,-0.725533,-1.078221,5.736223,-0.343539,-0.388720,0.740045,-0.858236,-0.170227,-0.575124,-0.700417,-0.854088,-0.725358,0.166017,-0.183001,1.645220,-0.321681,0.071049,-1.184848,-0.400543,-0.774815,10.467188,3.759741,-2.132777,6.828751,-3.882622,-0.012632,2.540224,0.909436,0.894902,0.643654,0.571186,0.710026,0.952778,0.716888,0.694228,0.721176,0.837320,1.083422,0.875582,1.210937,0.824143,0.873896,0.970839
Fold 3,7.201744,0.255087,-0.032151,-1.129055,0.097649,-0.973995,1.556766,4.263272,2.074270,5.948839,1.982965,-0.652972,-1.827341,7.583483,-0.930723,-0.476074,1.164423,-2.025478,-0.359240,-1.210601,-1.413115,-1.781033,-1.048064,1.065007,-0.902392,1.505582,-0.432042,0.645395,-2.029255,-0.812754,-0.822418,10.425203,3.676976,-2.475050,6.859202,-3.846473,-0.031882,2.178871,0.909436,0.781396,0.538932,0.535513,0.974724,0.951021,0.653899,0.717623,0.624753,0.963596,1.039735,1.158655,1.053839,0.645763,0.849183,0.977069
Fold 4,7.118850,-0.118362,-0.371504,-0.845635,-0.043549,-0.835140,1.695044,4.962766,1.452973,7.150771,2.231402,-0.831439,-1.865417,8.347602,-0.554145,-0.191494,1.551400,-2.692420,-2.421422,-1.138823,-1.550988,-2.603975,-1.595638,0.581491,-1.247336,1.603906,-0.411382,1.879966,-5.207752,-1.128642,-0.875179,10.633536,4.074087,-2.572915,6.722703,-3.790248,0.018087,2.554140,0.909436,0.930578,0.574611,0.688458,0.901315,0.999955,0.642758,0.644788,0.545748,1.002189,1.141475,1.036694,1.029036,0.645332,0.921416,0.912103
Fold 5,7.178907,0.451700,-0.193969,-1.359041,-0.034673,-0.407584,2.182018,5.374624,1.430862,7.660935,2.410786,-0.511798,-2.426799,8.870484,-0.741650,-0.438423,1.486931,-2.411572,-1.634192,-1.156137,-1.204491,-2.630658,-2.230507,0.315005,-1.998505,1.790806,-0.057924,-0.079634,-2.549875,-0.865560,-0.641928,10.932824,4.060351,-2.558109,6.973341,-3.787766,0.006205,2.485699,0.909436,0.981086,0.581097,0.697814,0.863638,1.000216,0.698247,0.644893,0.512153,0.989877,1.206975,1.019986,1.037286,0.634347,0.918059,0.883698
mean,7.191248,0.244338,-0.155210,-1.229525,0.014210,-0.697739,1.435658,3.951411,1.520707,5.745880,2.007430,-0.768233,-1.521854,7.365336,-0.692139,-0.388518,1.147070,-1.693861,-0.999421,-0.789475,-1.236924,-1.569647,-1.205133,0.652466,-0.808563,1.630571,-0.364799,0.684779,-2.538839,-0.813365,-0.795446,10.496872,3.774506,-2.294876,6.840396,-3.876951,0.004302,2.426672,0.909436,0.902349,0.580169,0.628177,0.877481,0.978501,0.667469,0.679628,0.588026,0.943516,1.130990,1.025620,1.084985,0.670395,0.892217,0.938884
std,0.058309,0.198829,0.120200,0.220417,0.054295,0.287098,0.505325,1.218140,0.318260,1.586020,0.331255,0.204057,0.701427,1.190853,0.218787,0.102468,0.337828,0.870637,0.879483,0.516007,0.291466,1.028016,0.642749,0.389460,0.801497,0.092480,0.178044,0.698634,1.405891,0.234375,0.084412,0.295581,0.284618,0.321933,0.080644,0.106500,0.025262,0.139112,0.000000,0.066544,0.034851,0.064354,0.091524,0.022133,0.034471,0.029548,0.076547,0.059381,0.061991,0.089998,0.066809,0.078483,0.027378,0.035726


### Stats

In [12]:
stats_np = np.array(stats)
losses_np = np.expand_dims(np.array(losses), 1)
columns = ["mse_train"] + ["mse_val"] + ["r2_val"] + ["rmse_val"] + ["r2/rmse_val"]
index = [f"Fold {i+1}" for i, _ in enumerate(stats_np)]  + ["mean"] + ["std"]
all_stats = np.hstack((losses_np, stats_np))
all_stats_mean = np.mean(all_stats, axis=0)
all_stats_std = np.std(all_stats, axis=0)
df = pd.DataFrame(np.vstack((all_stats, all_stats_mean, all_stats_std)),
                  columns=columns,
                  index=index)

display.display(df) 

,mse_train,mse_val,r2_val,rmse_val,r2/rmse_val
Fold 1,968.345229,563.167710,0.392779,23.731155,0.016551
Fold 2,822.100751,1118.673735,0.293750,33.446580,0.008783
Fold 3,897.381531,815.443027,0.309165,28.555963,0.010827
Fold 4,831.964562,1083.384766,0.295701,32.914811,0.008984
Fold 5,878.874540,881.446683,0.304579,29.689168,0.010259
mean,879.733323,892.423184,0.319195,29.667535,0.011081
std,52.474059,201.052845,0.037225,3.501504,0.002841


MD
